In [ ]:
import os
import pathlib
import pandas as pd

ARTIFACT_DIR = pathlib.Path(f"{os.getcwd()}/../artifacts/")
CSV_DIR = pathlib.Path(f"{os.getcwd()}/../data-raw/csv/")
assert pathlib.Path.exists(CSV_DIR), f"You must extract ZIP files to '{CSV_DIR}'"

In [ ]:
econ_df = pd.read_csv(CSV_DIR / "world_economy_freedom.csv")
elec_df = pd.read_csv(CSV_DIR / "global_electricity_production_data.csv")
water_df = pd.read_csv(CSV_DIR / "water.csv")

In [ ]:
print(econ_df.info())
econ_df.describe()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(econ_df["Country ID"], econ_df["2023 Score"])
plt.title("Economic Freedom Score - 2023")
plt.xlabel("Country ID")
plt.ylabel("Overall Score 2023")
plt.savefig(ARTIFACT_DIR / "EconScore2023.png")

In [ ]:
plt.scatter(econ_df["Change from 2022"], econ_df["2023 Score"])
plt.title("Economic Freedom - Change from 2022 vs Score 2023")
plt.xlabel("Change from 2022")
plt.ylabel("Score 2023")
plt.savefig(ARTIFACT_DIR / "Change-vs-Score2023.png")

In [ ]:
from scipy import stats
value = 70  # Above the p75
percentile = stats.percentileofscore(econ_df['2023 Score'], value)
print(f"The value {value} is at the {percentile:.2f} percentile.")
econ_df.sample(n=5)

In [ ]:
print(elec_df.info())
elec_df.describe()

In [ ]:
elec_df.head()

In [ ]:
elec_df.parameter.unique()

In [ ]:
elec_df["year"] = elec_df["date"].apply(lambda x: int(x.split("/")[-1]))
print(elec_df.year.unique())
elec_df[["country_name", "date", "year"]].head()

In [ ]:
elec_pivot = elec_df.pivot_table(index=["country_name", "year"], columns=["parameter", "product", "unit"], values="value")
print(elec_pivot.info())
elec_pivot.describe()

In [ ]:
elec_pivot.head(50)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Extract just the features and make a corr matrix.
corr = elec_pivot.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.savefig(ARTIFACT_DIR / "elec_corr.png")
plt.show()

In [ ]:
print(water_df.info())
water_df.describe()

In [ ]:
water_df.head(10)

In [ ]:
water_pivot = water_df.pivot_table(index=['REF_AREA:Geographic area', 'TIME_PERIOD:Time period'], 
                          columns='INDICATOR:Indicator', 
                          values='OBS_VALUE:Observation Value').reset_index()
print(water_pivot.info())
water_pivot.describe()

In [ ]:
water_indicators = [name.split(": ") for name in water_pivot.columns if name.startswith("WS_") and ":" in name]
for ind in water_indicators:
    print(f"{ind[0]:<12}: {ind[1]}")

In [ ]:
water_pivot.columns = [col.split(':')[0] for col in water_pivot.columns]
water_pivot.TIME_PERIOD = water_pivot.TIME_PERIOD.astype(int)
water_pivot.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Extract just the features and make a corr matrix.
corr = water_pivot[[ind[0] for ind in water_indicators]].corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
# Add the indicators and their definitions to the right side of the heatmap
for i, ind in enumerate(water_indicators):
    ax.text(x=1.2, y=i/len(water_indicators), s=f"{ind[0]}: {ind[1]}", 
            transform=ax.transAxes, verticalalignment='center')

plt.savefig(ARTIFACT_DIR / "water_corr.png")
plt.show()

In [ ]:
# Reset the column names and index to get a single level.
# Clone for idempotence.
flat_elec_pivot = elec_pivot.copy(deep=True)
flat_elec_pivot.columns = ['_'.join(col).strip() for col in elec_pivot.columns.values]
flat_elec_pivot.reset_index(inplace=True)
print(flat_elec_pivot.shape)
flat_elec_pivot.head()

In [ ]:
water_pivot.rename(columns={"REF_AREA": "country_name", "TIME_PERIOD": "year"}, inplace=True)
water_pivot.country_name = water_pivot.country_name.str.split(":").str[-1].str.strip()
print(water_pivot.shape)
water_pivot.head()

In [ ]:
econ_df = econ_df.rename(columns={"Country Name": "country_name"})
raw_features = (water_pivot[["country_name", "year", "WS_PPL_W-SM"]]
.merge(
        flat_elec_pivot[["country_name", "year", "Net Electricity Production_Electricity_GWh"]], 
        how="inner", 
        on=["country_name", "year"]
    ))
print(raw_features.shape)
print(raw_features.info())
raw_features

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(raw_features["WS_PPL_W-SM"], raw_features["Net Electricity Production_Electricity_GWh"])
plt.xlabel("WS_PPL_W-SM")
plt.ylabel("Net Electricity Production_Electricity_GWh")
plt.title("WS_PPL_W-SM vs Net Electricity Production_Electricity_GWh")
plt.savefig(ARTIFACT_DIR / "WaterSan-vs-Elec.png")
plt.show()

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

MISSING_SENTINEL = np.nan
COUNTRY = 0
YEAR = 1
WATER = 2
ELEC = 3

def calculate_deltas(data: pd.DataFrame) -> dict:
    # Values like { <country_name>: { <year>: [<water_delta>, <elec_delta>] } }
    acc = defaultdict(dict)
    records = data.to_records(index=False)
    
    prev_country_name = ""
    prev_year_water = MISSING_SENTINEL
    prev_year_elec = MISSING_SENTINEL
    
    for record in records:
        acc[record[COUNTRY]][record[YEAR]] = [np.nan, np.nan]
        
        if record[COUNTRY] != prev_country_name:
            prev_country_name = record[COUNTRY]
            prev_year_water = record[WATER]
            prev_year_elec = record[ELEC]
            continue
        
        if prev_year_water != MISSING_SENTINEL and not np.isnan(record[WATER]):
            acc[record[COUNTRY]][record[YEAR]][0] = ((record[WATER] - prev_year_water)/prev_year_water) * 100
        
        if prev_year_elec != MISSING_SENTINEL and not np.isnan(record[ELEC]):
            acc[record[COUNTRY]][record[YEAR]][1] = ((record[ELEC] - prev_year_elec)/prev_year_elec) * 100

        prev_year_water = record[WATER]
        prev_year_elec = record[ELEC]
            
    return acc

deltas = calculate_deltas(raw_features)
print(deltas["Hungary"])

In [ ]:
water_deltas = []
elec_deltas = []
for _, years in deltas.items():
    for _, pair in years.items():
        if np.isnan(pair[0]) or np.isnan(pair[1]):
            continue
        water_deltas.append(pair[0])
        elec_deltas.append(pair[1])
plt.scatter(water_deltas, elec_deltas)
plt.ylim(-20, 20)  # Some outliers.
plt.xlabel("Water Delta (YoY)")
plt.ylabel("Electricity Delta (YoY)")
plt.title("Changes in Water Sanitization vs Electricity Generation")
plt.savefig(ARTIFACT_DIR / "Changes-Water-vs-Elec.png")

In [ ]:
# FLATTEN - single row per country.
df_pivot = raw_features.pivot_table(index='country_name', columns='year')
# DELTAS to DataFrame.
deltas_df = pd.DataFrame(deltas).T
deltas_df.columns = pd.MultiIndex.from_tuples([(year, 'delta') for year in deltas_df.columns])
# UNPACK DELTAS.
for year in deltas_df.columns.get_level_values(0).unique():
    deltas_df[('delta_WS_PPL_W-SM', year)] = deltas_df[(year, 'delta')].apply(lambda x: x[0] if hasattr(x, '__iter__') else np.nan)
    deltas_df[('delta_Net Electricity Production_Electricity_GWh', year)] = deltas_df[(year, 'delta')].apply(lambda x: x[1] if hasattr(x, '__iter__') else np.nan)
# DROP OLD DELTAS OBJECTS.
deltas_df = deltas_df.drop(columns='delta', level=1)

In [ ]:
# MERGE flattened data with deltas information.
feature_df = pd.merge(df_pivot, deltas_df, left_index=True, right_index=True, how='outer')
# SORT and REPLACE missing values.
feature_df = feature_df.sort_index(axis=1).replace({None: np.nan})
# DROP known empty deltas columns.
feature_df = feature_df.drop(columns=[('delta_Net Electricity Production_Electricity_GWh', 2012), ('delta_WS_PPL_W-SM', 2012)])
# RESHAPE FOR EXPORT
feature_df.columns = ['_'.join([str(part) for part in col]).strip() for col in feature_df.columns.values]
feature_df.reset_index(inplace=True)
feature_df.rename(columns={'index': 'country_name'}, inplace=True)
# EXPORT
feature_df.to_csv(ARTIFACT_DIR / "feature_df.csv", index=False, sep="\t", na_rep='NaN')
feature_df.describe()

In [ ]:
# CREATE FULL DATASET
final_df = feature_df.merge(
    econ_df[["country_name", "Region", "Govt Integrity", "Gov\'t Spending", "Tax Burden", "2022 Score", "2023 Score", "Change from 2022"]], 
    on="country_name", how="left"
)
columns = {
        col: "_".join([part for part in col.replace("-", "_").replace("'", "").split(" ")])
        for col in final_df.columns
    }
final_df.rename(columns=columns, inplace=True)
final_df.to_csv(ARTIFACT_DIR / "final_data.csv", index=False, sep="\t", na_rep='NaN')
final_df.sample(n=3)

In [ ]:
import re
interesting_cols = [c for c in final_df.columns if re.match(r".+_20[1,2][0-9]", c)] + ["Govt_Integrity", "Govt_Spending", "Tax_Burden"]
corr = final_df[interesting_cols].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.savefig(ARTIFACT_DIR / "feature_corr.png")
plt.show()